In [914]:
from networkx import maximum_flow
from networkx import gnm_random_graph
from networkx import draw 
from networkx import adjacency_matrix
from networkx import all_simple_paths
from networkx import shortest_path
from networkx import set_edge_attributes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy

# Graph

In [956]:
min_weight = 5
max_weight = 15
n_nodes = 5 * 10 ** 4
n_edges = 10 ** 6
# n_nodes, n_edges = 50000, 1000000 # <|-- takes 17.5 s to compute the maximum flow in a graph
source, destination = 0, 1 # don't need to randomize (graph is randomized)

## Creates a random valuated graph

In [957]:
def modify_weights_graph (Graph, ctr_lists, new_weights) :
    """ utility function for setting weights (dual problem of shortest path) """
    set_edge_attributes(Graph, 0, name="weight")

    set_edge_attributes(
        Graph,
        values = dict({edge : dual for edge, dual in zip (ctr_lists, new_weights)}),
        name = 'weight'
    )

In [958]:
Graph = gnm_random_graph(n_nodes, n_edges, directed=True)
Graph_dual = deepcopy(Graph)

for (u, v) in Graph.edges():
    Graph.edges[u,v]['weight'] = np.random.uniform(min_weight, max_weight)

In [959]:
Adj_matrix = adjacency_matrix(Graph)
Adj_matrix.shape

C:\Users\val\AppData\Local\Temp\ipykernel_13556\2680887831.py:1: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Adj_matrix = adjacency_matrix(Graph)


(50000, 50000)

In [960]:
flow, flow_dict = maximum_flow(Graph, _s = source, _t = destination, capacity='weight')
# total flow, flow first node
flow, flow_dict[0]

(142.43062558494952,
 {7297: 13.175234886960148,
  19529: 7.740546264129416,
  26352: 10.062570958821341,
  38758: 11.872549180021304,
  37947: 10.120692593800516,
  18004: 8.113392099264876,
  44419: 4.106951130080311,
  19836: 3.9366031279385787,
  4700: 6.401647632216307,
  43326: 10.972908689924274,
  15468: 5.0697878846611095,
  8286: 8.947865116398008,
  2892: 11.22300375240323,
  43059: 6.199137461117241,
  43244: 12.374703440239706,
  4698: 7.2346272370855065,
  24453: 14.840650451513348})

Maximum flow is equivalent to maximum path generation **if we consider all possible paths**

# Column Generation

In [961]:
import pulp as pl

In [962]:
solver = pl.getSolver('CPLEX_CMD', timeLimit=10)
solver.path

'cplex.exe'

In [963]:
def get_n_paths (Graph, source, destination, max_iter = 16):
    """ Get the max_iter paths of all_simple_paths generator: Graph: source --|> destination """
    list_ = []
    for (index, path) in enumerate(all_simple_paths(Graph, source, destination, cutoff=10)):
        list_.append(path)
        if (index >= max_iter - 1):
            return (list_)
    return (list_)

In [964]:
# affichage de n chemins 
len(get_n_paths(Graph, source, destination, max_iter=50))

50

## Master

In [965]:
# Etape 0 INIT THE MASTER PROBLEM

lpProb = pl.LpProblem("path_generation", pl.LpMaximize)
lpProb += 0 # initiate the objective function to 0 (necessary line of code)

In [967]:
# useful to know the final paths at the end
path2idx = {}
ctr_lists = []
path_count = 0

In [968]:
def add_paths (lpProb, paths, path_count, path2idx):
    # pour chaque path ajouter une variable et ses coef. dans les contraintes
    for path in paths :

        # add a var path
        prov = pl.LpVariable("X"+str(path_count), 0, max_weight, pl.LpContinuous)
        lpProb.objective.addterm(prov, 1)
        
        # for each edge on the path, search the associated constraint (indexed by edge)
        for index in range(len(path)-1) :
            index_ctr = str((path[index], path[index + 1]))
            if index_ctr in lpProb.constraints:
                lpProb.constraints[index_ctr].addterm(prov, 1) # add term in constraint
            else:
                lpProb.constraints[index_ctr] = prov <= Graph.edges[(path[index], path[index + 1])]
                ctr_lists.append((path[index], path[index + 1]))
        
        path2idx[path_count] = str(path)
        path_count += 1
    return (path_count, path2idx)

In [969]:
path_count, path2idx = add_paths (lpProb, get_n_paths (Graph, source, destination, max_iter = 1), path_count, path2idx)

In [970]:
# solver = pl.CPLEX_PY()
result = lpProb.solve()
# result = lpProb.solve(pl.GUROBI_CMD(options=[("MIPgap", 0.9)]))

## Dual variables

In [972]:
def get_duals(lpPb):
    """ Get the dual variables from a problem Pulp """
    return [c.pi for _, c in list(lpPb.constraints.items())]

## Iterative procedure

In [973]:
go_on = True

while go_on :
# for i in range(100):

    duals = get_duals(lpProb)

    modify_weights_graph (Graph_dual, ctr_lists, duals)

    path_q = shortest_path(Graph_dual, source, destination, weight="weight")

    reduced_cost = 1

    for e in range(len(path_q)-1) :
        try :
            reduced_cost -= lpProb.constraints[str((path_q[e], path_q[e + 1]))].pi
        except KeyError : # <|-- constraint does not exist (so dual == 0)
            pass

    if reduced_cost > 0 : # add path q in lp Problem
        path_count, path2idx = add_paths(lpProb, [path_q], path_count, path2idx)
        lpProb.solve()
    else :
        go_on = False

In [974]:
flow = 0
for var in lpProb.variables():
    flow += var.value()
print(flow)

142.43062568999997
